<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
《<a href="http://mng.bz/orYv">从零构建大语言模型</a>》一书的补充代码，作者：<a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>代码仓库：<a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# 内存高效的模型权重加载

- 本notebook提供了在GPU（或CPU）内存有限时加载大型预训练或微调模型的技巧
- 具体来说，它专注于以下情况：使用`torch.save(model.state_dict(), "model.pth")`保存了模型（例如在第5-7章中），并希望在以后的新会话中加载它以继续预训练或进行额外的微调
- 虽然该示例使用了LLM，但本笔记本中解释的方法是通用的，适用于加载任何PyTorch模型，不仅限于LLM

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/memory-efficient-loading/memory-efficient-loading.webp" width="800px">

In [1]:
from importlib.metadata import version

pkgs = [
    "torch",
]
for p in pkgs:
    print(f"{p} version: {version(p)}")

torch version: 2.6.0


## 1. 基准测试工具

- 首先，定义一些用于跟踪VRAM（GPU内存）的工具代码
- 稍后，还将介绍一个跟踪主系统RAM（CPU内存）的工具
- 稍后应用这些函数时，它们的作用就会变得清晰

In [2]:
import gc
import time
import torch


def start_memory_tracking():
    """Initialize GPU memory tracking."""
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats()
    else:
        print("This notebook is intended for CUDA GPUs but CUDA is not available.")

def print_memory_usage():
    max_gpu_memory = torch.cuda.max_memory_allocated() / (1024 ** 3)  # Convert bytes to GB
    print(f"Maximum GPU memory allocated: {max_gpu_memory:.1f} GB")

def cleanup():
    gc.collect()
    torch.cuda.empty_cache()
    time.sleep(3)  # some buffer time to allow memory to clear
    torch.cuda.reset_peak_memory_stats()
    max_memory_allocated = torch.cuda.max_memory_allocated(device) / (1024 ** 3)
    print(f"Maximum GPU memory allocated: {max_memory_allocated:.1f} GB")

## 2. 模型设置

- 此代码部分设置模型本身
- 在这里，使用"large" GPT-2模型来使示例更有趣（可以使用"gpt2-small (124M)"来降低内存需求和本笔记本的执行时间）

In [3]:
from previous_chapters import GPTModel
# If the `previous_chapters.py` file is not available locally,
# you can import it from the `llms-from-scratch` PyPI package.
# For details, see: https://github.com/rasbt/LLMs-from-scratch/tree/main/pkg
# E.g.,
# from llms_from_scratch.ch04 import GPTModel



BASE_CONFIG = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.0,        # Dropout rate
    "qkv_bias": True         # Query-key-value bias
}

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

CHOOSE_MODEL = "gpt2-xl (1558M)"

BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

- 现在，看看GPU内存函数的作用

In [4]:
start_memory_tracking()


model = GPTModel(BASE_CONFIG)
device = torch.device("cuda")
model.to(device)

print_memory_usage()

Maximum GPU memory allocated: 6.4 GB


- 此外，确保模型正常运行，通过传入一些示例张量进行测试

In [5]:
# Test if the model works (no need to track memory here)
test_input = torch.tensor([[1, 2, 3]]).to(device)
model.eval()

with torch.no_grad():
    model(test_input)

- 接下来，假设正在预训练模型并将其保存供以后使用
- 为简单起见，跳过实际的预训练过程，只保存初始化的模型（但应用相同的概念）

In [6]:
# Training code would go here...

model.train()
torch.save(model.state_dict(), "model.pth")

- 最后，在Python会话中删除模型和示例张量以重置GPU内存

In [7]:
del model, test_input
cleanup()

Maximum GPU memory allocated: 0.0 GB


## 3. 权重加载

- 现在加载预训练模型权重
- 看看加载之前保存的模型需要多少GPU内存

In [8]:
# Then load pretrained weights

start_memory_tracking()

model = GPTModel(BASE_CONFIG)
model.to(device)

model.load_state_dict(
    torch.load("model.pth", map_location=device, weights_only=True)
)
model.to(device)
model.eval();

print_memory_usage()

Maximum GPU memory allocated: 12.8 GB


- 注意，内存是前一个会话的两倍大
- 这是因为在短时间内将相同的模型在内存中保存了两次：
  - 第一次通过`model.to(device)`
  - 第二次通过代码行`model.load_state_dict(torch.load("model.pth", map_location=device, weights_only=True))`；最终，加载的模型权重将被复制到模型中，`state_dict`将被丢弃，但在短暂的时间内，既有主模型又有加载的`state_dict`在内存中
- 后续部分专注于解决这个问题
- 但首先，测试模型并重置GPU内存

In [9]:
# Test if the model works (no need to track memory here)
test_input = torch.tensor([[1, 2, 3]]).to(device)
model.eval()

with torch.no_grad():
    model(test_input)

del model, test_input
cleanup()

Maximum GPU memory allocated: 0.0 GB


## 4. 顺序加载权重

- 对于前一部分强调的GPU内存中存在两次模型权重的问题，一种解决方法是顺序加载模型
- 在下面：
  - 首先将模型加载到GPU内存中
  - 然后将模型权重加载到CPU内存中
  - 最后逐个将每个参数复制到GPU内存中

In [10]:
start_memory_tracking()

model = GPTModel(BASE_CONFIG).to(device)

state_dict = torch.load("model.pth", map_location="cpu", weights_only=True)

print_memory_usage()

# Sequentially copy weights to the model's parameters
with torch.no_grad():
    for name, param in model.named_parameters():
        if name in state_dict:
            param.copy_(state_dict[name].to(device))
        else:
            print(f"Warning: {name} not found in state_dict.")

print_memory_usage()

Maximum GPU memory allocated: 6.4 GB
Maximum GPU memory allocated: 6.7 GB


- 如上所示，可以看到内存使用量比之前低得多
- 注意内存从6.4增加到6.7GB，因为最初只有模型在内存中，然后有模型加1个参数张量在内存中（暂时将参数张量移动到GPU以便可以使用`".to"`分配给它）
- 总的来说，这是一个显著的改进
- 同样，简要测试模型，然后为下一节重置GPU内存

In [11]:
# Test if the model works (no need to track memory here)
test_input = torch.tensor([[1, 2, 3]]).to(device)
model.eval()

with torch.no_grad():
    model(test_input)

del model, test_input, state_dict, param
cleanup()

Maximum GPU memory allocated: 0.0 GB


## 5. 使用低CPU内存加载模型

- 在之前的会话中，通过先将权重（`state_dict`）加载到CPU内存中，然后逐个复制到模型中来减少GPU内存使用
- 然而，如果CPU内存是有限的怎么办？
- 本节使用PyTorch所谓的"meta"设备方法在GPU内存大但CPU内存小的机器上加载模型
- 但首先，定义一个方便函数来监控CPU内存

In [12]:
import os
import psutil
from threading import Thread


def memory_usage_in_gb(func, *args, **kwargs):
    process = psutil.Process(os.getpid())

    # Measure the baseline memory usage before running the function
    baseline_mem = process.memory_info().rss / 1024 ** 3  # in GB

    # Start monitoring memory in a separate thread
    mem_usage = []
    done = False

    def monitor_memory():
        while not done:
            mem_usage.append(process.memory_info().rss / 1024 ** 3)  # Convert to GB
            time.sleep(0.1)

    t = Thread(target=monitor_memory)
    t.start()

    # Run the function
    func(*args, **kwargs)

    # Stop monitoring
    done = True
    t.join()

    peak_mem_usage_gb = max(mem_usage) - baseline_mem
    return peak_mem_usage_gb


- 首先，跟踪上一节中顺序权重加载方法的CPU内存

In [13]:
def load_sequentially():
    start_memory_tracking()

    model = GPTModel(BASE_CONFIG).to(device)

    state_dict = torch.load("model.pth", map_location="cpu", weights_only=True)

    print_memory_usage()

    # Sequentially copy weights to the model's parameters
    with torch.no_grad():
        for name, param in model.named_parameters():
            if name in state_dict:
                param.copy_(state_dict[name].to(device))
            else:
                print(f"Warning: {name} not found in state_dict.")

    print_memory_usage()


peak_memory_used = memory_usage_in_gb(load_sequentially)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

Maximum GPU memory allocated: 6.4 GB
Maximum GPU memory allocated: 6.7 GB
-> Maximum CPU memory allocated: 6.3 GB


- 现在，假设有一台CPU内存少但GPU内存大的机器
- 可以通过引入PyTorch所谓的"meta"设备来权衡CPU内存和GPU内存的使用
- PyTorch的meta设备是一种特殊的设备类型，允许创建张量而不为其数据分配实际内存，有效创建"meta"张量
- 这对于需要张量形状和类型但不需要内存分配开销的任务（如模型分析或架构定义）很有用

In [14]:
def load_sequentially_with_meta():
    start_memory_tracking()

    with torch.device("meta"):
        model = GPTModel(BASE_CONFIG)

    model = model.to_empty(device=device)

    state_dict = torch.load("model.pth", map_location=device, weights_only=True)

    print_memory_usage()

    # Sequentially copy weights to the model's parameters
    with torch.no_grad():
        for name, param in model.named_parameters():
            if name in state_dict:
                param.copy_(state_dict[name])
            else:
                print(f"Warning: {name} not found in state_dict.")

    print_memory_usage()

peak_memory_used = memory_usage_in_gb(load_sequentially_with_meta)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

Maximum GPU memory allocated: 12.8 GB
Maximum GPU memory allocated: 12.8 GB
-> Maximum CPU memory allocated: 1.3 GB


- 如上所示，通过在meta设备上创建模型并直接将权重加载到GPU内存中，有效地降低了CPU内存需求
- 有人可能会问："那么顺序权重加载仍然必要吗，这与原始方法相比如何？"
- 检查简单的PyTorch权重加载方法作为比较（来自本notebook中第一个权重加载部分）

In [15]:
def baseline():
    start_memory_tracking()

    model = GPTModel(BASE_CONFIG)
    model.to(device)

    model.load_state_dict(torch.load("model.pth", map_location=device, weights_only=True))
    model.to(device)
    model.eval();

    print_memory_usage()

peak_memory_used = memory_usage_in_gb(baseline)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

Maximum GPU memory allocated: 12.8 GB
-> Maximum CPU memory allocated: 4.4 GB


- 如上所示，不使用meta设备的"简单"权重加载方法使用更多内存
- 换句话说，如果有一台CPU内存有限的机器，可以使用meta设备方法直接加载模型权重到GPU内存中以减少峰值CPU内存使用

## 6. 使用 `mmap=True`（推荐）

- 作为中级或高级`torch.load`用户，可能想知道这些方法与PyTorch中的`mmap=True`设置相比如何
- PyTorch中的`mmap=True`设置启用内存映射文件I/O，允许张量直接从磁盘存储访问数据，从而通过在RAM有限时不将整个文件加载到RAM中来减少内存使用
- 另请参阅[mikaylagawarecki](https://github.com/rasbt/LLMs-from-scratch/issues/402)的有用评论
- 初看之下，它可能看起来不如上面的顺序方法高效

In [37]:
def best_practices():
  with torch.device("meta"):
      model = GPTModel(BASE_CONFIG)

  model.load_state_dict(
      torch.load("model.pth", map_location=device, weights_only=True, mmap=True),
      assign=True
  )

  print_memory_usage()

peak_memory_used = memory_usage_in_gb(best_practices)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

Maximum GPU memory allocated: 6.4 GB
-> Maximum CPU memory allocated: 5.9 GB


- CPU RAM使用量如此之高的原因是这台机器上有足够的CPU RAM可用
- 但是，如果在CPU RAM有限的机器上运行此方法，`mmap`方法将使用更少的内存

## 7. 其他方法

- 本notebook专注于在PyTorch中加载权重的简单、内置方法
- 对于CPU内存有限的情况，推荐的方法是已经充分解释的`mmap=True`方法
- 另一种选择是暴力方法，分别保存和加载每个权重张量

In [13]:
model = GPTModel(BASE_CONFIG)
# Assume `model` is your trained model
state_dict = model.state_dict()

# Create a directory to store individual parameter files
os.makedirs("model_parameters", exist_ok=True)

# Save each parameter tensor separately
for name, param in state_dict.items():
    torch.save(param.cpu(), f"model_parameters/{name}.pt")

del model

In [16]:
def load_individual_weights():

    start_memory_tracking()

    with torch.device("meta"):
        model = GPTModel(BASE_CONFIG)

    model = model.to_empty(device=device)

    print_memory_usage()
    param_dir = "model_parameters"

    with torch.no_grad():
        for name, param in model.named_parameters():
            weight_path = os.path.join(param_dir, f"{name}.pt")
            if os.path.exists(weight_path):
                param_data = torch.load(weight_path, map_location="cpu", weights_only=True)
                param.copy_(param_data)
                del param_data  # Free memory
            else:
                print(f"Warning: {name} not found in {param_dir}.")

    print_memory_usage()


peak_memory_used = memory_usage_in_gb(load_individual_weights)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

Maximum GPU memory allocated: 6.4 GB
Maximum GPU memory allocated: 6.4 GB
-> Maximum CPU memory allocated: 0.3 GB
